# Semantic Kernel Agent Solution

## Prepare the files

In [1]:
import os

file_directory = "../Data/products"

# Get our files in the file directory
try:
    filenames = os.listdir(file_directory)
    print(filenames)
except FileNotFoundError:
    print(f"Directory '{file_directory}' not found.")

def get_filepath_for_filename(filename: str) -> str:
    base_directory = file_directory
    return os.path.join(base_directory, filename)



['product_info_19.pdf', 'product_info_13.pdf', 'product_info_9.pdf', 'product_info_7.pdf', 'product_info_10.pdf', 'product_info_3.pdf', 'product_info_8.pdf', 'product_info_4.pdf', 'product_info_14.pdf', 'product_info_20.pdf', 'product_info_17.pdf', 'product_info_1.pdf', 'product_info_5.pdf', 'product_info_6.pdf', 'product_info_16.pdf', 'product_info_18.pdf', 'product_info_12.pdf', 'product_info_15.pdf', 'product_info_11.pdf', 'product_info_2.pdf']


## Create a Plugin

In [2]:
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from typing import Annotated

# Define a sample plugin for the hiking company
class HikingPlugin:
    """A sample Hiking Plugin for the hiking company."""

    @kernel_function(description="Provides a list of hiking destinations.")
    def get_destinations(self) -> Annotated[str, "Returns the hiking destinations that you can explore."]:
        return """
            United States
            Australia
            France
        """

    @kernel_function(description="Provides the typical hiking budget for a destination")
    def get_hiking_budget(self, 
        destination: Annotated[str, "The name of the hiking destination"]) -> Annotated[str, "Returns the budget for the destination."]:
        if destination == "United States":
            return """
                Budget Hikers: Around $50 per day. This includes camping, cooking your own meals, and using public transportation.
                Mid-Range Hikers: Approximately $150 per day. This covers mid-range accommodations, dining at average restaurants, and some paid attractions.
                Luxury Hikers: About $400 per day. This includes luxury lodges, fine dining, and private transportation.
            """
        elif destination == "Australia":
            return """
                Budget Hikers: Around $40 per day. This includes camping, cooking your own meals, and using public transport.
                Mid-Range Hikers: Approximately $120 per day. This covers mid-range accommodations, dining at average restaurants, and some paid attractions.
                Luxury Hikers: About $350 per day. This includes luxury lodges, fine dining, and private transportation.
            """
        elif destination == "France":
            return """
                Budget Hikers: Around $45 per day. This includes camping, cooking your own meals, and using public transportation.
                Mid-Range Hikers: Approximately $130 per day. This covers mid-range accommodations, dining at average restaurants, and some paid attractions.
                Luxury Hikers: About $380 per day. This includes luxury lodges, fine dining, and private transportation.            
            """
        else:
            return "Destination not found"
    
    @kernel_function(description="Provides the weather for a hiking destination")
    def get_weather(self, 
        destination: Annotated[str, "The name of the hiking destination"]) -> Annotated[str, "Returns the weather for the destination."]:
        if destination == "United States":
            return """
                The United States has a diverse climate. In general:
                - **Winter**: Cold in the north, mild in the south. Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.
                - **Summer**: Hot and humid in the south, dry in the west. Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).
            """
        elif destination == "Australia":
            return """
                Australia experiences varied climates:
                - **Winter**: Mild in the north, cooler in the south. Average temperatures range from 46°F (8°C) in Canberra to 77°F (25°C) in Darwin.
                - **Summer**: Hot and dry in the interior, humid in the north. Average temperatures range from 68°F (20°C) in Hobart to 91°F (33°C) in Darwin.
            """
        elif destination == "France":
            return """
                France has several climate zones:
                - **Winter**: Cold in the north, mild in the south. Average temperatures range from 35.5°F (2°C) in the northeast to 48°F (9°C) in the south.
                - **Summer**: Warm and sunny. Average temperatures range from 63°F (17°C) in the north to 77°F (25°C) along the Mediterranean coast.
            """
        else:
            return "Destination not found"

## Reformat citations with the proper filenames

In [3]:
async def reformat_citations(agent, response):
    from semantic_kernel.contents import StreamingAnnotationContent

    # Extract the annotations
    annotations = [item for item in response.items if isinstance(item, StreamingAnnotationContent)]

    # Convert the response content to a string
    paragraph = str(response.content)

    # Dictionary to store key-value pairs of text and filename
    text_filename_pairs = {}

    # Iterate over the annotations and extract the relevant information
    for annotation in annotations:
        file_id = annotation.file_id
        text = annotation.quote
        # Retrieve the filename from the file_id
        cited_file = await agent.client.files.retrieve(file_id)
        filename = cited_file.filename

        if text not in text_filename_pairs:
            text_filename_pairs[text] = []
        text_filename_pairs[text].append(filename)

    # Replace the citation texts with their corresponding filenames prefixed with " Source: "
    for text, filenames in text_filename_pairs.items():
        sources = " Source: " + ", ".join(filenames)
        paragraph = paragraph.replace(text, sources)

    return paragraph

## Create an Agent and Thread

In [4]:
from semantic_kernel.agents import AssistantAgentThread, AzureAssistantAgent
from semantic_kernel.contents import StreamingAnnotationContent

# Create the client using Azure OpenAI resources and configuration
client, model = AzureAssistantAgent.setup_resources()

# Upload the files to the client
file_ids: list[str] = []
for path in [get_filepath_for_filename(filename) for filename in filenames]:
    with open(path, "rb") as file:
        file = await client.files.create(file=file, purpose="assistants")
        file_ids.append(file.id)

vector_store = await client.vector_stores.create(
    name="assistant_search",
    file_ids=file_ids,
)

# Get the file search tool and resources
file_search_tools, file_search_tool_resources = AzureAssistantAgent.configure_file_search_tool(vector_store_ids=vector_store.id)

# Create the assistant definition
definition = await client.beta.assistants.create(
    model=model,
    instructions="""
            You are provided a document store and tools to search for information.
            The document store contains Hiking products of Contoso company.
            Always analyze the document store and tools to provide an answer to the user's question.
            Never rely on your knowledge of information not included in the document store and tools.
            Always format response using markdown.
        """,
    name="SampleAssistantAgent",
    tools=file_search_tools,
    tool_resources=file_search_tool_resources,
)

# Create the agent using the client and the assistant definition
agent = AzureAssistantAgent(
    client=client,
    definition=definition,
    plugins=[HikingPlugin()],
)

# Create a thread for the agent
thread: AssistantAgentThread = None

## Helper Function


In [5]:
async def run_agent(user_question, thread):
   
    async for response in agent.invoke_stream(messages=user_question, thread=thread):
        thread = response.thread
        annotations = [item for item in response.items if isinstance(item, StreamingAnnotationContent)]
        #Print the Assistant response
        if annotations is None:
            print(f"{response.content}", end="", flush=True)
        else:
            print(f"{await reformat_citations(agent,response)}", end="", flush=True)

In [6]:
user_question = "What is the price of the SummitClimber Backpack?"
await run_agent(user_question, thread)

The price of the SummitClimber Backpack is $120 Source: product_info_9.pdf.

## Appending Messages to the Thread

In [10]:
user_question = "What are the list of countries I can hike?"
await run_agent(user_question, thread)

You can hike in the following countries:

- United States
- Australia
- France

In [11]:
user_question = "What is the weather in Australia?"
await run_agent(user_question, thread)

The weather in Australia varies significantly across different regions:

- **Winter**: Mild in the north and cooler in the south.
  - Average temperatures range from 46°F (8°C) in Canberra to 77°F (25°C) in Darwin.
  
- **Summer**: Hot and dry in the interior, humid in the north.
  - Average temperatures range from 68°F (20°C) in Hobart to 91°F (33°C) in Darwin.

In [8]:
user_question = "If I go to the US, how much money do I need for a 3 day hike and what would the weather be like?"
await run_agent(user_question, thread)

Based on the information provided:

### Budget
- **Budget Hikers:** Around $50 per day. This includes camping, cooking your own meals, and using public transportation.
- **Mid-Range Hikers:** Approximately $150 per day. This covers mid-range accommodations, dining at average restaurants, and some paid attractions.
- **Luxury Hikers:** About $400 per day. This includes luxury lodges, fine dining, and private transportation.

For a 3-day hike:
- **Budget Option:** $150
- **Mid-Range Option:** $450
- **Luxury Option:** $1200

### Weather
The United States has a diverse climate. In general:
- **Winter:** Cold in the north, mild in the south. Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.
- **Summer:** Hot and humid in the south, dry in the west. Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).

For specific destinations within the US and exact weather conditions during your intended hike dates, it would be best to obtain local weat

In [9]:
user_question = """
I have a total budget of $1000. 
If I will hike in the US for 3 days on a budget, what products can you recommend that I buy with the money left?
"""
await run_agent(user_question, thread)

To help you stay within your budget for a 3-day hike in the US, I suggest the following products from Contoso's hiking gear collection. Based on the total budget remaining after accounting for trip expenses (approximately $1000 - $180 for three days = $820), these products will provide you with durable and reliable equipment for your hiking adventure:

1. **TrailWalker Hiking Shoes** - Price: $110
   - Durable and waterproof construction, cushioned insole, supportive midsole, and a breathable mesh lining Source: product_info_11.pdf.

2. **SummitClimber Backpack** - Price: $120
   - Capacity: 60 liters, lightweight nylon material, ergonomic design, adjustable hip belt, integrated rain cover, and multiple compartments Source: product_info_9.pdf.

3. **SkyView 2-Person Tent** - Price: $200
   - Durable and waterproof, spacious interior, quick and easy setup, multiple storage pockets, and compact design Source: product_info_15.pdf.

4. **TrailBlaze Hiking Pants** - Price: $75
   - Made of 

## Deleting Files, Thread, Agent

In [12]:
if agent is not None:
    [await client.files.delete(file_id) for file_id in file_ids]
    await thread.delete() if thread else None
    await client.beta.assistants.delete(agent.id)
